# Welchのt検定かマン・ホイットニーのU検定か

# 概要

- 使用する検定を決めるために正規分布の確認をするプログラムです．
- Step1では，各群の**歪度**計算と**シャピロ・ウィルク検定**を行います．
    - 歪度は，右裾が長いか左に偏った分布のときには正の値を，左裾が長いか右に偏った分布のときには負の値をとります．左右対称の分布の場合には0になります．
    - シャピロ・ウィルク検定は，データが正規分布しているかどうかを検定する手法です．検定の結果，p<0.05で有意となったときは「正規分布しない」と判断し，p≧0.05となったときは“正規分布していない，とはいえない（＝正規分布している）”とみなします．
- Step2では，Step1の結果からWelchのt検定，マン・ホイットニーのU検定，ブルンナー・ムンツェル法のいずれかを行います．
    - Welchのt検定は，データが正規分布しているという前提で行うパラメトリック手法です．
    - マン・ホイットニーのU検定は，データが正規分布していなくても良いノンパラメトリック手法です．しかし，分布が比較的類似していることが条件となります．
    - ブルンナー・ムンツェル法はノンパラメトリック手法であり，分布がバラついていても使用できる手法です．
- 視覚的な確認のために，箱ひげ図，ヒストグラム，QQプロットを描画します．

# 入力

```jsx
file_path = '分析したいExcelファイル名.xlsx'
selected_sheet = 分析したいExcelシート番号
automated_stat_decision(df, '比較したい項目のカラム名', '比較したい群を含むカラム名', '群1', '群2')
```

# 出力

- **箱ひげ図**：外れ値と中央値の確認
- ヒストグラム：分布の形の確認
- QQプロット：正規性の確認

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import japanize_matplotlib

In [ ]:
def automated_stat_decision(df, target_col, group_col, group1_val, group2_val):
    # --- 1. データ準備 ---
    try:
        # 文字列型・数値型の揺らぎ吸収
        data1 = df[df[group_col] == group1_val][target_col].dropna()
        data2 = df[df[group_col] == group2_val][target_col].dropna()
        
        if len(data1) == 0 and len(data2) == 0:
            # 型変換トライ
            g1_alt = str(group1_val) if isinstance(group1_val, int) else int(group1_val)
            g2_alt = str(group2_val) if isinstance(group2_val, int) else int(group2_val)
            data1 = df[df[group_col] == g1_alt][target_col].dropna()
            data2 = df[df[group_col] == g2_alt][target_col].dropna()
            if len(data1) > 0:
                group1_val, group2_val = g1_alt, g2_alt

    except Exception as e:
        print(f"データ抽出エラー: {e}")
        return

    n1, n2 = len(data1), len(data2)
    print(f"{'='*70}")
    print(f" 自動判定プロセス: {target_col}")
    print(f" 群1({group1_val}): N={n1}, 群2({group2_val}): N={n2}")
    print(f"{'='*70}")

    if n1 < 3 or n2 < 3:
        print("エラー: N数が少なすぎるため(N<3)、統計的な正規性判定が不能です。")
        print("推奨: 無条件でノンパラメトリック検定を使用してください。")
        return

    # --- 2. 可視化 (箱ひげ図, ヒストグラム, Q-Qプロット) ---
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # (1) 箱ひげ図 (外れ値・中央値チェック)
    plot_df = pd.concat([
        pd.DataFrame({target_col: data1, group_col: group1_val}),
        pd.DataFrame({target_col: data2, group_col: group2_val})
    ])
    sns.boxplot(data=plot_df, x=group_col, y=target_col, ax=axes[0, 0], palette="Set2")
    axes[0, 0].set_title("1. Boxplot (Check Outliers)")

    # (2) ヒストグラム (分布の形)
    sns.histplot(data=plot_df, x=target_col, hue=group_col, kde=True, ax=axes[0, 1], palette="Set2", element="step")
    axes[0, 1].set_title("2. Histogram (Check Shape)")

    # (3) Q-Qプロット (正規性の視覚確認)
    stats.probplot(data1, dist="norm", plot=axes[1, 0])
    axes[1, 0].set_title(f"3. Q-Q Plot: {group1_val}")
    stats.probplot(data2, dist="norm", plot=axes[1, 1])
    axes[1, 1].set_title(f"3. Q-Q Plot: {group2_val}")

    plt.tight_layout()
    plt.show()

    # --- 3. 数値的な正規性チェック (Skewness & Shapiro-Wilk) ---
    print("\n【Step 1: 分布特性のチェック】")
    
    # 歪度 (Skewness)
    skew1 = stats.skew(data1)
    skew2 = stats.skew(data2)
    
    # Shapiro-Wilk検定
    sw_stat1, sw_p1 = stats.shapiro(data1)
    sw_stat2, sw_p2 = stats.shapiro(data2)
    
    # 判定フラグ
    # 基準: 歪度が絶対値2未満 かつ Shapiro p >= 0.05 なら「正規分布に近い」とみなす
    is_normal_1 = (abs(skew1) < 2.0) and (sw_p1 >= 0.05)
    is_normal_2 = (abs(skew2) < 2.0) and (sw_p2 >= 0.05)
    both_normal = is_normal_1 and is_normal_2

    print(f"  群1({group1_val}): 歪度={skew1:.2f}, Shapiro p={sw_p1:.4f} -> {'正規' if is_normal_1 else '非正規疑い'}")
    print(f"  群2({group2_val}): 歪度={skew2:.2f}, Shapiro p={sw_p2:.4f} -> {'正規' if is_normal_2 else '非正規疑い'}")

    # --- 4. 検定の実行と推奨 ---
    print(f"\n{'-'*70}")
    print("【Step 2: 分析結果と推奨】")
    
    # Cohen's d 計算関数
    def calc_cohens_d(x, y):
        nx, ny = len(x), len(y)
        dof = nx + ny - 2
        pool_std = np.sqrt(((nx-1)*np.var(x, ddof=1) + (ny-1)*np.var(y, ddof=1)) / dof)
        return (np.mean(x) - np.mean(y)) / pool_std

    # すべての検定を実行しておく
    # A. Welch's t-test
    t_stat, t_p = stats.ttest_ind(data1, data2, equal_var=False)
    d_val = calc_cohens_d(data1, data2)
    
    # B. Mann-Whitney U
    u_stat, u_p = stats.mannwhitneyu(data1, data2, alternative='two-sided')
    # r (効果量)
    n_total = n1 + n2
    z_val = (u_stat - (n1 * n2 / 2)) / np.sqrt(n1 * n2 * (n_total + 1) / 12)
    r_val = abs(z_val) / np.sqrt(n_total)
    
    # C. Brunner-Munzel (分布=t指定)
    bm_stat, bm_p = stats.brunnermunzel(data1, data2, distribution='t', alternative='two-sided')

    # --- 判定ロジック ---
    if both_normal:
        # ケース1: 正規分布とみなせる場合
        print(">> 判定: 分布の歪みが小さく、正規性が仮定できます。")
        print("   ★ 推奨: 【Welchのt検定】を主分析として採用してください。")
        print("\n   [主分析結果: Welch's t-test]")
        print(f"   t値: {t_stat:.3f}, p値: {t_p:.4f}, 効果量(d): {d_val:.3f}")
        print(f"   (判定: {'有意差あり' if t_p < 0.05 else '有意差なし'})")
        
        print("\n   [確認用: Mann-Whitney U]")
        print(f"   p値: {u_p:.4f} (t検定の結果と矛盾しないか確認)")

    else:
        # ケース2: 正規性が疑われる場合
        print(">> 判定: 歪度が大きい、またはShapiro検定で有意(p<0.05)です。")
        print("   ★ 推奨: 【ノンパラメトリック検定】を主分析として採用してください。")
        
        # 分散チェック (Brunner-Munzelを使うべきか判断するため)
        lev_stat, lev_p = stats.levene(data1, data2, center='median')
        if lev_p < 0.05 or (max(n1, n2)/min(n1, n2) > 2): # 分散が違う or N数比が大きい
            print("   (特に分散不均一、またはN数が不揃いなため、Brunner-Munzelを強く推奨します)")
            print("\n   [主分析結果: Brunner-Munzel検定]")
            print(f"   W値: {bm_stat:.3f}, p値: {bm_p:.4f}")
        else:
            print("\n   [主分析結果: Mann-Whitney U検定]")
            print(f"   U値: {u_stat:.1f}, p値: {u_p:.4f}, 効果量(r): {r_val:.3f}")

        print(f"   (判定: {'有意差あり' if (bm_p if lev_p < 0.05 else u_p) < 0.05 else '有意差なし'})")

        print("\n   [参考: Welch's t-test]")
        print(f"   p値: {t_p:.4f} (あくまで参考値です)")

    print(f"{'='*70}")

In [ ]:
# Excelファイルのパスを指定
file_path = 'AdvanceData.xlsx'

In [ ]:
xls = pd.ExcelFile(file_path)
print("--- シート一覧 ---")
for i, name in enumerate(xls.sheet_names):
    print(f"{i}: {name}")

In [ ]:
selected_sheet = 0  # ここを変更してシートを選択
df = pd.read_excel(file_path, sheet_name=selected_sheet)
print(f"\n選択されたシート '{xls.sheet_names[selected_sheet] if isinstance(selected_sheet, int) else selected_sheet}' を読み込みました。")

In [ ]:
automated_stat_decision(df, 'env', 'role', 'ケアスタッフ', 'パート')